In [4]:
import yaml
from qiskit_metal import Dict

def create_qubit_config(num_qubits, output_file='qubit_config.yaml'):
    """Create YAML configuration with list values."""
    
    # Base configuration with lists
    config = {
        'qubits': {
            'width': ['1000um'] * num_qubits,
            'height': ['1000um'] * num_qubits,
            'cross_width': ['60um'] * num_qubits,
            'cross_length': ['400um'] * num_qubits,
            'cpw_width': ['10um'] * num_qubits,
            'cpw_gap': ['6um'] * num_qubits,
            'jogsw_width': ['20um'] * num_qubits,
            'jogsw_length': ['20um'] * num_qubits,
            'pocket_width': ['2000um'] * num_qubits,
            'pocket_height': ['2000um'] * num_qubits,
            'squid_width': ['1um'] * num_qubits,
            'squid_height': ['1um'] * num_qubits,
            'squid_gap': ['0.5um'] * num_qubits,
            'connection_pads': {
                'readout': {
                    'connector_type': ['claw'] * num_qubits,
                    'claw_length': ['30um'] * num_qubits,
                    'ground_spacing': ['5um'] * num_qubits,
                    'claw_width': ['10um'] * num_qubits,
                    'claw_gap': ['6um'] * num_qubits,
                    'claw_cpw_length': ['40um'] * num_qubits
                },
                'bus': {
                    'connector_type': ['claw'] * num_qubits,
                    'claw_length': ['30um'] * num_qubits,
                    'ground_spacing': ['5um'] * num_qubits,
                    'claw_width': ['10um'] * num_qubits,
                    'claw_gap': ['6um'] * num_qubits,
                    'claw_cpw_length': ['40um'] * num_qubits
                }
            }
        },
        'resonators': {
            'width': ['10um'] * num_qubits,
            'gap': ['6um'] * num_qubits,
            'length': ['4000um'] * num_qubits,
            'fillet': ['100um'] * num_qubits,
            'coupling_length': ['200um'] * num_qubits,
            'coupling_gap': ['6um'] * num_qubits,
            'inductor_width': ['5um'] * num_qubits,
            'inductor_gap': ['3um'] * num_qubits
        }
    }

    with open(output_file, 'w') as f:
        yaml.dump(config, f, default_flow_style=False, sort_keys=False)
    
    return Dict(config)

def load_config_by_index(config_file, index):
    """Load configuration for specific index."""
    with open(config_file, 'r') as f:
        config = yaml.safe_load(f)
    
    single_config = {}
    # Extract values at specified index for each parameter
    for component in ['qubits', 'resonators']:
        single_config[component] = {}
        for param, values in config[component].items():
            if isinstance(values, dict):
                single_config[component][param] = {
                    k: {sk: sv[index] for sk, sv in v.items()}
                    for k, v in values.items()
                }
            else:
                single_config[component][param] = values[index]
    
    return Dict(single_config)

if __name__ == "__main__":
    config = create_qubit_config(3, 'qubit_config.yaml')
    print("Configuration saved to qubit_config.yaml")

Configuration saved to qubit_config.yaml


In [6]:
import yaml
from qiskit_metal import Dict

def extract_qubit_config(config_dict, index):
    """Extract single qubit configuration from list format."""
    single_config = {}
    for key, value in config_dict.items():
        if isinstance(value, list):
            single_config[key] = value[index]
        elif isinstance(value, dict):
            single_config[key] = extract_qubit_config(value, index)
    return single_config

def load_config(yaml_file, index=None):
    """Load YAML config file with option to extract specific qubit."""
    with open(yaml_file, 'r') as f:
        config = yaml.safe_load(f)
    
    if index is not None:
        # Extract single qubit config
        result = {}
        for component, params in config.items():
            result[component] = extract_qubit_config(params, index)
        return Dict(result)
    
    return Dict(config)

# Example usage
if __name__ == "__main__":
    # Load specific qubit
    qubit_0 = load_config('qubit_config.yaml', index=0)
    print(f"Qubit 0 width: {qubit_0.qubits.width}")
    
    # Load all qubits
    all_qubits = load_config('qubit_config.yaml')
    print(f"All qubit widths: {all_qubits.qubits.width}")

Qubit 0 width: 1000um
All qubit widths: ['1000um', '1000um', '1000um']


In [7]:
qubit_0

{'qubits': {'width': '1000um',
  'height': '1000um',
  'cross_width': '60um',
  'cross_length': '400um',
  'cpw_width': '10um',
  'cpw_gap': '6um',
  'jogsw_width': '20um',
  'jogsw_length': '20um',
  'pocket_width': '2000um',
  'pocket_height': '2000um',
  'squid_width': '1um',
  'squid_height': '1um',
  'squid_gap': '0.5um',
  'connection_pads': {'readout': {'connector_type': 'claw',
    'claw_length': '30um',
    'ground_spacing': '5um',
    'claw_width': '10um',
    'claw_gap': '6um',
    'claw_cpw_length': '40um'},
   'bus': {'connector_type': 'claw',
    'claw_length': '30um',
    'ground_spacing': '5um',
    'claw_width': '10um',
    'claw_gap': '6um',
    'claw_cpw_length': '40um'}}},
 'resonators': {'width': '10um',
  'gap': '6um',
  'length': '4000um',
  'fillet': '100um',
  'coupling_length': '200um',
  'coupling_gap': '6um',
  'inductor_width': '5um',
  'inductor_gap': '3um'}}

In [8]:
import yaml
from qiskit_metal import Dict

def update_qubit_params(yaml_file, qubit_index, updates, output_file=None):
    """
    Update parameters for a specific qubit
    
    Args:
        yaml_file (str): Input YAML file path
        qubit_index (int): Index of qubit to update
        updates (dict): Dictionary of parameters to update
        output_file (str, optional): Output file path. If None, overwrites input
    """
    # Load existing config
    with open(yaml_file, 'r') as f:
        config = yaml.safe_load(f)
    
    # Update parameters
    for param, value in updates.items():
        if param in config['qubits']:
            config['qubits'][param][qubit_index] = value
    
    # Save updated config
    output_file = output_file or yaml_file
    with open(output_file, 'w') as f:
        yaml.dump(config, f, default_flow_style=False)
    
    return Dict(config)

# Example usage:
if __name__ == "__main__":
    # Update parameters for qubit 0
    updates = {
        'width': '1500um',
        'cross_width': '80um',
        'squid_gap': '0.8um'
    }
    
    updated_config = update_qubit_params(
        'qubit_config.yaml',
        qubit_index=0,
        updates=updates
    )

In [10]:
# Update specific qubit parameters
updates = {'width': '1550um', 'cross_width': '80um'}
update_qubit_params('qubit_config.yaml', qubit_index=0, updates=updates)

{'qubits': {'connection_pads': {'bus': {'claw_cpw_length': ['40um',
     '40um',
     '40um'],
    'claw_gap': ['6um', '6um', '6um'],
    'claw_length': ['30um', '30um', '30um'],
    'claw_width': ['10um', '10um', '10um'],
    'connector_type': ['claw', 'claw', 'claw'],
    'ground_spacing': ['5um', '5um', '5um']},
   'readout': {'claw_cpw_length': ['40um', '40um', '40um'],
    'claw_gap': ['6um', '6um', '6um'],
    'claw_length': ['30um', '30um', '30um'],
    'claw_width': ['10um', '10um', '10um'],
    'connector_type': ['claw', 'claw', 'claw'],
    'ground_spacing': ['5um', '5um', '5um']}},
  'cpw_gap': ['6um', '6um', '6um'],
  'cpw_width': ['10um', '10um', '10um'],
  'cross_length': ['400um', '400um', '400um'],
  'cross_width': ['80um', '60um', '60um'],
  'height': ['1000um', '1000um', '1000um'],
  'jogsw_length': ['20um', '20um', '20um'],
  'jogsw_width': ['20um', '20um', '20um'],
  'pocket_height': ['2000um', '2000um', '2000um'],
  'pocket_width': ['2000um', '2000um', '2000um'],
 